In [16]:
import numpy as np
import math
from nsepy import get_history
from datetime import date, datetime
import os
import tradingview_ta
from tradingview_ta import TA_Handler, Interval, Exchange
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import io
import sys
import telegram #TELEGRAM BOT
import requests
import sys
chat_id_group = "-1002245236166" #Group where Telegram will receive updates, had to add that BOT in that group as admin
TOKEN = "7419116752:AAFTGAcg9uuvl_j7andYX_r_kJfmgsxIKCo"
#Suppressing Console Output
original_stdout = sys.stdout
sys.stdout = None
sys.stdout = original_stdout
df = pd.read_csv("ind_nifty100list.csv")
stocks = df['Symbol'].unique().tolist() #Sending Output string
stocks[12] = 'BAJAJ_AUTO'
stocks[62] = 'M_M'
# INTERVAL_1_MINUTE = "1m", INTERVAL_5_MINUTES = "5m", INTERVAL_15_MINUTES = "15m", INTERVAL_30_MINUTES = "30m", INTERVAL_1_HOUR = "1h", INTERVAL_2_HOURS = "2h", INTERVAL_4_HOURS = "4h", INTERVAL_1_DAY = "1d", INTERVAL_1_WEEK = "1W", INTERVAL_1_MONTH = "1M"
values_list = []
def sendTeleAlert(message, chat_id):
 url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
 response = requests.get(url)
def posSz(entry_price, stoploss, target):
 try:
     posSz = math.floor(abs(float(200 / (entry_price - stoploss))))
     print(f'Position sizing for Max Loss 200 is {posSz} \n')
     values_list.append(posSz)  # 0
 except ZeroDivisionError:
     print('Position sizing not found as Entry Price equals Stoploss')
def riskreward(entry_price, stoploss, target):
 num = float(abs(entry_price - target) / abs(entry_price - stoploss))
 string = f'1:{num}'
 values_list.append(num) #1
def stocks_calc(entry_price, stoploss, target):
 total_caps = [50000, 55000, 60000]
 for i in range(len(total_caps)):
  num_stocks = math.floor(float(total_caps[i]/ entry_price))
  total_capital_used = float(entry_price * num_stocks)
  targetgains = float(abs(entry_price - target) * num_stocks)
  maxloss = float(abs(entry_price - stoploss) * num_stocks)
  values_list.append(num_stocks) #2,6,10
  values_list.append(total_capital_used) #3,7,11
  values_list.append(targetgains) #4,8,12
  values_list.append(maxloss) #5,9,13
while True:
 for i in range(len(stocks)):
     ssw = TA_Handler(
         symbol=stocks[i],
         screener="india",
         exchange="NSE",
         interval=Interval.INTERVAL_15_MINUTES            #TIMEFRAME
         )
     rec = ssw.get_analysis()
     FETCH_TIME = rec.time.strftime("%H:%M:%S")
     HIGH = rec.indicators["high"]
     LOW = rec.indicators["low"]
     BBLOWER = rec.indicators["BB.lower"]
     BBHIGHER = rec.indicators["BB.upper"]
     SMA = rec.indicators["SMA20"]
     if ( LOW > BBHIGHER and HIGH > BBHIGHER): #Upperregion case i.e. SELL Case
       print(f"SELL CASE: {ssw.symbol}, Time: {FETCH_TIME}")
       {posSz(LOW, HIGH, SMA)}
       {riskreward(LOW, HIGH, SMA)}
       {stocks_calc(LOW, HIGH, SMA)}
       print({i})
       message = f"For Piyush \n\nSELL CASE: {ssw.symbol}, Time: {FETCH_TIME},\n\n Entry Price: {LOW}\n Stoploss: {HIGH}\n Target: {SMA}\n Risk-Reward is 1:{values_list[1]} \n\n Position Sizing is {values_list[0]} stocks for Max Loss of 200 and corresponding Total Capital is {LOW*values_list[0]} with Target Gains of {values_list[0]*(LOW-SMA)} \n\n Total Capital Used is {values_list[3]} \n Number of stocks is {values_list[2]} \n Target Gains = {values_list[4]} \n Max Loss is = {values_list[5]} \n\n Total Capital Used is {values_list[7]} \n Number of stocks is {values_list[6]} \n Target Gains = {values_list[8]} \n Max Loss is = {values_list[9]} \n\n Total Capital Used is {values_list[11]} \n Number of stocks is {values_list[10]} \n Target Gains = {values_list[12]} \n Max Loss is = {values_list[13]} \n\n"
       print(message)
       sendTeleAlert(message, chat_id_group)
       values_list.clear()
     if ( LOW < BBLOWER and HIGH < BBLOWER): #Lowerregion case i.e. BUY Case
       print(f"BUY CASE: {ssw.symbol}, Time: {FETCH_TIME}")
       {posSz(HIGH, LOW, SMA)}
       {riskreward(HIGH, LOW, SMA)}
       {stocks_calc(HIGH, LOW, SMA)}
       message = f"For Piyush \n\nBUY CASE: {ssw.symbol}, Time: {FETCH_TIME},\n\n Entry Price: {HIGH}\n Stoploss: {LOW}\n Target: {SMA}\n Risk-Reward is 1:{values_list[1]} \n\n Position Sizing is {values_list[0]} stocks for Max Loss of 200 and corresponding Total Capital is {HIGH*values_list[0]} with Target Gains of {values_list[0]*(SMA-HIGH)} \n\n Total Capital Used is {values_list[3]} \n Number of stocks is {values_list[2]} \n Target Gains = {values_list[4]} \n Max Loss is = {values_list[5]} \n\n Total Capital Used is {values_list[7]} \n Number of stocks is {values_list[6]} \n Target Gains = {values_list[8]} \n Max Loss is = {values_list[9]} \n\n Total Capital Used is {values_list[11]} \n Number of stocks is {values_list[10]} \n Target Gains = {values_list[12]} \n Max Loss is = {values_list[13]} \n\n"
       print(message)
       sendTeleAlert(message, chat_id_group)
       values_list.clear()
     else:
       print(f"No result in {ssw.symbol}")
     i += 1

No result in ABB
No result in ADANIENSOL
No result in ADANIENT
No result in ADANIGREEN
No result in ADANIPORTS
No result in ADANIPOWER
No result in ATGL
No result in AMBUJACEM
No result in APOLLOHOSP
No result in ASIANPAINT
No result in DMART
No result in AXISBANK
No result in BAJAJ_AUTO
No result in BAJFINANCE
No result in BAJAJFINSV
No result in BAJAJHLDNG
No result in BANKBARODA
No result in BERGEPAINT
No result in BEL
No result in BPCL
No result in BHARTIARTL
No result in BOSCHLTD
No result in BRITANNIA
No result in CANBK
No result in CHOLAFIN
No result in CIPLA
No result in COALINDIA
No result in COLPAL
No result in DLF
No result in DABUR
No result in DIVISLAB
No result in DRREDDY
No result in EICHERMOT
No result in GAIL
No result in GODREJCP
No result in GRASIM
No result in HCLTECH
No result in HDFCBANK
No result in HDFCLIFE
No result in HAVELLS
No result in HEROMOTOCO
No result in HINDALCO
No result in HAL
No result in HINDUNILVR
No result in ICICIBANK
No result in ICICIGI
No re

KeyboardInterrupt: 